In [0]:
# # Run cell on cluster restart or if receiving error: 
# # AttributeError: 'EMR' object has no attribute 'create_persistent_app_ui'

# %pip install --upgrade boto3 botocore
# %restart_python

## EMR Spark Event Log Analyzer
 
This script analyzes EMR clusters to extract performance metrics from Spark History Servers. It discovers EMR clusters, connects to their persistent Spark History Server UIs, fetches application, job, stage, and SQL query data, and then processes this information into Spark DataFrames for performance analysis and optimization insights.

## Required IAM Permissions
#### EMR
  - elasticmapreduce:ListClusters
  - elasticmapreduce:DescribeCluster
  - elasticmapreduce:ListSteps
  - elasticmapreduce:DescribeStep
  - elasticmapreduce:CreatePersistentAppUI
  - elasticmapreduce:GetPersistentAppUIPresignedURL
  - elasticmapreduce:ListInstanceGroups
  - elasticmapreduce:ListInstanceFleets
#### S3
  - s3:PutObject
  - s3:GetObject
  - s3:ListBucket
#### STS
  - sts:GetCallerIdentity



In [0]:
import datetime
from datetime import timedelta
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# ----------------------------------------------------------------------
# Configuration Parameters
# ----------------------------------------------------------------------
# Parse and validate configuration
dbutils.widgets.text("aws_region", "us-east-1", "AWS Region")
dbutils.widgets.text("emr_cluster_arn", "", "EMR Cluster ARN (optional - leave blank to discover clusters)")
dbutils.widgets.text("timeout_seconds", "300", "Request Timeout (seconds)")
dbutils.widgets.text("max_applications", "50", "Max Applications to Analyze per Cluster")
dbutils.widgets.dropdown("environment", "dev", ["dev", "prod"], "Environment (dev/prod)")
dbutils.widgets.text("s3_output_path", "", "S3 Output Path (prod only)")

# Updated: cluster_states dropdown with new options
dbutils.widgets.dropdown("cluster_states", "TERMINATED,WAITING", ["TERMINATED", "WAITING", "TERMINATED,WAITING", "ALL"], "EMR Cluster States to Analyze")

dbutils.widgets.text("cluster_name_filter", "", "Cluster Name Filter (optional - partial name match)")
dbutils.widgets.text("max_clusters", "5", "Max Clusters to Analyze")

# New: Widgets for CreatedAfter and CreatedBefore dates
# Format: YYYY-MM-DD. Leave blank for no date filter.
dbutils.widgets.text("created_after_date", "", "EMR Clusters Created After (YYYY-MM-DD)")
dbutils.widgets.text("created_before_date", "", "EMR Clusters Created Before (YYYY-MM-DD)")

# New: Parameters for timeout and concurrency
dbutils.widgets.text("persistent_ui_timeout_seconds", "180", "Persistent App UI Timeout (seconds)")
dbutils.widgets.text("max_cluster_threads", "50", "Max Concurrent Cluster Analysis Threads")
dbutils.widgets.text("max_app_threads", "100", "Max Concurrent App Analysis Threads per Cluster")


AWS_REGION = dbutils.widgets.get("aws_region").strip() or "us-east-1"
EMR_CLUSTER_ARN = dbutils.widgets.get("emr_cluster_arn").strip()
TIMEOUT_SECONDS = int(dbutils.widgets.get("timeout_seconds") or "300")
MAX_APPLICATIONS = int(dbutils.widgets.get("max_applications") or "50")
CLUSTER_STATES = dbutils.widgets.get("cluster_states").strip()
CLUSTER_NAME_FILTER = dbutils.widgets.get("cluster_name_filter").strip()
MAX_CLUSTERS = int(dbutils.widgets.get("max_clusters") or "5")
ENVIRONMENT = dbutils.widgets.get("environment").strip()
S3_OUTPUT_PATH = dbutils.widgets.get("s3_output_path").strip()
PERSISTENT_UI_TIMEOUT_SECONDS = int(dbutils.widgets.get("persistent_ui_timeout_seconds") or "180")
MAX_CLUSTER_THREADS = int(dbutils.widgets.get("max_cluster_threads") or "50")
MAX_APP_THREADS = int(dbutils.widgets.get("max_app_threads") or "100")


# Date parameters
if ENVIRONMENT == "dev":
    CREATED_AFTER_DATE_STR = dbutils.widgets.get("created_after_date").strip()
    CREATED_BEFORE_DATE_STR = dbutils.widgets.get("created_before_date").strip()
    PARSED_CREATED_AFTER_DATE = None
    if CREATED_AFTER_DATE_STR:
        try:
            PARSED_CREATED_AFTER_DATE = datetime.datetime.strptime(CREATED_AFTER_DATE_STR, "%Y-%m-%d")
        except ValueError as e:
            logger.error("❌ Invalid format for created_after_date: %s. Expected YYYY-MM-DD.", CREATED_AFTER_DATE_STR, exc_info=True)
            raise ValueError(f"Invalid format for created_after_date: {CREATED_AFTER_DATE_STR}. Expected YYYY-MM-DD.") from e
    PARSED_CREATED_BEFORE_DATE = None
    if CREATED_BEFORE_DATE_STR:
        try:
            PARSED_CREATED_BEFORE_DATE = datetime.datetime.strptime(CREATED_BEFORE_DATE_STR, "%Y-%m-%d") + timedelta(days=1, seconds=-1)
        except ValueError as e:
            logger.error("❌ Invalid format for created_before_date: %s. Expected YYYY-MM-DD.", CREATED_BEFORE_DATE_STR, exc_info=True)
            raise ValueError(f"Invalid format for created_before_date: {CREATED_BEFORE_DATE_STR}. Expected YYYY-MM-DD.") from e
    # Validate date range
    if PARSED_CREATED_AFTER_DATE and PARSED_CREATED_BEFORE_DATE and PARSED_CREATED_AFTER_DATE >= PARSED_CREATED_BEFORE_DATE:
        logger.error("❌ created_after_date (%s) cannot be on or after created_before_date (%s).", CREATED_AFTER_DATE_STR, CREATED_BEFORE_DATE_STR, exc_info=True)
        raise ValueError("created_after_date cannot be on or after created_before_date.")
else:
    # In prod, analyze only the last 24 hours
    PARSED_CREATED_BEFORE_DATE = datetime.datetime.now()
    PARSED_CREATED_AFTER_DATE = PARSED_CREATED_BEFORE_DATE - timedelta(days=1)
    CREATED_AFTER_DATE_STR = PARSED_CREATED_AFTER_DATE.strftime("%Y-%m-%d")
    CREATED_BEFORE_DATE_STR = PARSED_CREATED_BEFORE_DATE.strftime("%Y-%m-%d")


# Updated: Parse cluster states based on the new dropdown options
if CLUSTER_STATES == "ALL":
    # Comprehensive list of all EMR cluster states for 'ALL' option
    CLUSTER_STATES_LIST = [
        'STARTING', 'BOOTSTRAPPING', 'RUNNING', 'WAITING',
        'TERMINATING', 'TERMINATED', 'TERMINATED_WITH_ERRORS'
    ]
elif CLUSTER_STATES == "TERMINATED":
    CLUSTER_STATES_LIST = ['TERMINATED']
elif CLUSTER_STATES == "WAITING":
    CLUSTER_STATES_LIST = ['WAITING']
elif CLUSTER_STATES == "TERMINATED,WAITING":
    CLUSTER_STATES_LIST = ['TERMINATED', 'WAITING']
else:
    # Default to TERMINATED and WAITING for any unexpected value
    CLUSTER_STATES_LIST = ['TERMINATED', 'WAITING']
    logger.warning("Invalid cluster_states value '%s'. Defaulting to ['TERMINATED', 'WAITING'].", CLUSTER_STATES)


print("Configuration:")
print(f" Environment: {ENVIRONMENT}")
print(f" AWS Region: {AWS_REGION}")
print(f" EMR Cluster ARN: {EMR_CLUSTER_ARN or 'Auto-discover clusters'}")
print(f" Timeout: {TIMEOUT_SECONDS} seconds")
print(f" Max Applications per Cluster: {MAX_APPLICATIONS}")
print(f" Cluster States to Analyze: {CLUSTER_STATES_LIST}")
print(f" Cluster Name Filter: {CLUSTER_NAME_FILTER or 'None'}")
print(f" Max Clusters to Analyze: {MAX_CLUSTERS}")
print(f" Created After Date: {CREATED_AFTER_DATE_STR or 'None'}")
print(f" Created Before Date: {CREATED_BEFORE_DATE_STR or 'None'}")
print(f" S3 Output Path: {S3_OUTPUT_PATH or 'None (dev mode or not specified)'}")
print(f" Persistent UI Timeout: {PERSISTENT_UI_TIMEOUT_SECONDS} seconds")
print(f" Max Cluster Threads: {MAX_CLUSTER_THREADS}")
print(f" Max App Threads: {MAX_APP_THREADS}")


Configuration:
 Environment: dev
 AWS Region: us-east-1
 EMR Cluster ARN: Auto-discover clusters
 Timeout: 300 seconds
 Max Applications per Cluster: 10
 Cluster States to Analyze: ['STARTING', 'BOOTSTRAPPING', 'RUNNING', 'WAITING', 'TERMINATING', 'TERMINATED', 'TERMINATED_WITH_ERRORS']
 Cluster Name Filter: None
 Max Clusters to Analyze: 10
 Created After Date: None
 Created Before Date: None
 S3 Output Path: s3://ttd-vertica-backups/env=test/vertica-ext/databricks-stage/sparkmetrics
 Persistent UI Timeout: 180 seconds
 Max Cluster Threads: 50
 Max App Threads: 50


### AWS Boto3 Helper Functions

In [0]:
"""
EMR Persistent App UI Client
This module provides functionality to create an EMR Persistent App UI, retrieve its details and presigned URL,
and establish an HTTP session with proper cookie management for Spark History Server access.
"""

import logging
import time
from typing import Dict, Optional, Tuple
from urllib.parse import urlparse

import boto3
import requests
from botocore.exceptions import ClientError

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


class ServerConfig:
    """Configuration class for EMR Persistent UI client."""

    def __init__(self, emr_cluster_arn: str, timeout: int = 300):
        """
        Initialize ServerConfig.

        :param emr_cluster_arn: The EMR cluster ARN
        :param timeout: Request timeout in seconds
        :raises ValueError: If emr_cluster_arn is invalid
        """
        if not emr_cluster_arn or not emr_cluster_arn.startswith("arn:aws:elasticmapreduce:"):
            raise ValueError("Invalid EMR cluster ARN format")
        self.emr_cluster_arn = emr_cluster_arn
        self.timeout = timeout


class EMRPersistentUIClient:
    """Client for managing EMR Persistent App UI and HTTP sessions."""

    def __init__(self, server_config: ServerConfig):
        """
        Initialize the EMR client.

        :param server_config: ServerConfig object
        """
        self.emr_cluster_arn = server_config.emr_cluster_arn
        self.region = self.emr_cluster_arn.split(":")[3] # Extract region from ARN
        # Initialize boto3 client with credentials
        self.emr_client = boto3.client(
            "emr",
            region_name=self.region
        )
        self.session = requests.Session()
        self.persistent_ui_id: Optional[str] = None
        self.presigned_url: Optional[str] = None
        self.base_url: Optional[str] = None
        self.timeout: int = server_config.timeout
        self.presigned_url_ready: bool = False

    def create_persistent_app_ui(self) -> Dict:
        """
        Create a persistent app UI for the given cluster.

        :returns: Response from create-persistent-app-ui API call
        :raises ClientError: If the API call fails
        """
        logger.info("Creating persistent app UI for cluster: %s", self.emr_cluster_arn)
        try:
            response = self.emr_client.create_persistent_app_ui(
                TargetResourceArn=self.emr_cluster_arn
            )
            self.persistent_ui_id = response.get("PersistentAppUIId")
            runtime_role_enabled = response.get("RuntimeRoleEnabledCluster", False)
            logger.info("✅ Persistent App UI created successfully")
            logger.info(" Persistent UI ID: %s", self.persistent_ui_id)
            logger.info(" Runtime Role Enabled: %s", runtime_role_enabled)
            return response
        except ClientError as e:
            error_code = e.response["Error"]["Code"]
            error_message = e.response["Error"]["Message"]
            logger.error(
                "❌ Failed to create persistent app UI: %s - %s", error_code, error_message, exc_info=True
            )
            raise
        except Exception as e:
            logger.error("❌ Unexpected error creating persistent app UI: %s", str(e), exc_info=True)
            raise

    def get_presigned_url(self, ui_type: str = "SHS") -> str:
        """
        Get presigned URL for the persistent app UI.

        :param ui_type: Type of UI ('SHS' for Spark History Server)
        :returns: Presigned URL string
        :raises ValueError: If no persistent UI ID is available
        :raises ClientError: If the API call fails
        """
        if not self.persistent_ui_id:
            raise ValueError("No persistent UI ID available. Create one first.")
        logger.info(
            "Getting presigned URL for persistent app UI: %s (type: %s)", self.persistent_ui_id, ui_type
        )
        try:
            response = self.emr_client.get_persistent_app_ui_presigned_url(
                PersistentAppUIId=self.persistent_ui_id,
                PersistentAppUIType=ui_type
            )
            self.presigned_url_ready = response.get("PresignedURLReady")
            self.presigned_url = response.get("PresignedURL")
            # Extract base URL from presigned URL
            parsed_url = urlparse(self.presigned_url)
            self.base_url = f"{parsed_url.scheme}://{parsed_url.netloc}/shs"
            logger.info("✅ Presigned URL obtained successfully")
            logger.info(" Base URL: %s", self.base_url)
            return self.presigned_url
        except ClientError as e:
            error_code = e.response["Error"]["Code"]
            error_message = e.response["Error"]["Message"]
            logger.error(
                "❌ Failed to get presigned URL: %s - %s", error_code, error_message, exc_info=True
            )
            raise
        except Exception as e:
            logger.error("❌ Unexpected error getting presigned URL: %s", str(e), exc_info=True)
            raise

    def setup_http_session(self) -> requests.Session:
        """
        Set up HTTP session with proper headers and cookie management.

        :returns: Configured requests.Session object
        :raises ValueError: If no presigned URL is available
        """
        if not self.presigned_url:
            raise ValueError("No presigned URL available. Get one first.")
        logger.info("Setting up HTTP session with cookie management")
        # Configure session with appropriate headers
        self.session.headers.update(
            {
                "User-Agent": "EMR-Persistent-UI-Client/1.0",
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                "Accept-Language": "en-US,en;q=0.5",
                "Accept-Encoding": "gzip, deflate",
                "Connection": "keep-alive",
                "Upgrade-Insecure-Requests": "1"
            }
        )
        try:
            # Make initial request to establish session and get cookies
            logger.info("Making initial request")
            response = self.session.get(
                self.presigned_url, timeout=self.timeout, allow_redirects=True
            )
            response.raise_for_status()
            logger.info("✅ HTTP session established successfully")
            logger.info(" Status Code: %s", response.status_code)
            logger.info(" Cookies: %s cookie(s) stored", len(self.session.cookies))
            # Log cookie details (without sensitive values)
            for cookie in self.session.cookies:
                logger.debug(" Cookie: %s (domain: %s)", cookie.name, cookie.domain)
            return self.session
        except requests.exceptions.RequestException as e:
            logger.error("❌ Failed to establish HTTP session: %s", str(e), exc_info=True)
            raise
        except Exception as e:
            logger.error("❌ Unexpected error setting up HTTP session: %s", str(e), exc_info=True)
            raise

    def initialize(self, max_wait_time: int) -> Tuple[str, requests.Session]:
        """
        Initialize the EMR Persistent UI client by creating a persistent app UI,
        polling until it is ready, getting a presigned URL, and setting up an HTTP session.

        :param max_wait_time: The maximum time in seconds to wait for the persistent UI to become ready.
        :returns: Tuple containing the base URL and configured session
        :raises ValueError: If the persistent UI does not become ready within the timeout period
        """
        # Step 1: Create persistent app UI
        self.create_persistent_app_ui()

        # Step 2: Poll for the presigned URL until it's ready
        wait_interval = 10  # Check every 10 seconds
        total_waited = 0
        url_is_ready = False

        logger.info("Waiting for Persistent App UI to become ready...")

        while total_waited < max_wait_time:
            try:
                # Directly call the boto3 client to check the ready status without failing
                response = self.emr_client.get_persistent_app_ui_presigned_url(
                    PersistentAppUIId=self.persistent_ui_id, PersistentAppUIType="SHS"
                )
                url_is_ready = response.get("PresignedURLReady", False)
            except ClientError as e:
                # This can happen if the UI is not yet fully initialized; we just wait and retry.
                logger.warning(
                    "Could not check for presigned URL, will retry. Error: %s", str(e)
                )

            if url_is_ready:
                logger.info("✅ Persistent App UI is ready.")
                break

            logger.info(
                "Persistent App UI not ready yet. Waiting %s seconds before retrying...",
                wait_interval
            )
            time.sleep(wait_interval)
            total_waited += wait_interval

        # After waiting, check if we succeeded
        if not url_is_ready:
            raise ValueError(
                f"Persistent App UI did not become ready after waiting {total_waited} seconds."
            )

        # Step 3: Get the actual presigned URL and setup its properties (now that we know it's ready)
        self.get_presigned_url()

        # Step 4: Setup HTTP session
        self.setup_http_session()

        return self.base_url, self.session


### EMR Cluster Discovery

In [0]:
from datetime import datetime, timedelta
from typing import List, Dict, Optional # Added Optional
from botocore.exceptions import ClientError


class EMRClusterDiscovery:
    """Discovery and management of EMR clusters."""


    def __init__(self, region: str):
        """
        Initialize the EMR cluster discovery client.


        :param region: AWS region
        :raises TypeError: If region is not a non-empty string.
        """
        if not isinstance(region, str) or not region:
            raise TypeError("AWS region must be a non-empty string.")
        self.region = region
        self.emr_client = boto3.client("emr", region_name=region)


    def discover_clusters(self,
                          states: Optional[List[str]] = None,
                          name_filter: Optional[str] = None,
                          max_clusters: int = 10,
                          created_after: Optional[datetime] = None,   # Modified: New parameter
                          created_before: Optional[datetime] = None) -> List[Dict]: # Modified: New parameter
        """
        Discover EMR clusters based on criteria.


        :param states: List of cluster states to filter by. Defaults to ['TERMINATED', 'WAITING'].
                       Refer to boto3 EMR documentation for valid states.
        :param name_filter: Partial name to filter clusters. Optional.
        :param max_clusters: Maximum number of clusters to return. Defaults to 10.
        :param created_after: datetime object. Only return clusters created after this time. Optional.
        :param created_before: datetime object. Only return clusters created before this time. Optional.
        :returns: List of cluster summaries, each a dictionary containing cluster metadata.
        :raises TypeError: If inputs are not of the correct type.
        :raises ValueError: If input values are out of acceptable ranges (e.g., max_clusters < 1).
        :raises ClientError: If AWS API call fails.
        """
        # Input validation
        if not isinstance(states, (list, type(None))):
            raise TypeError(":param states: must be a list of strings or None.")
        if states:
            for state in states:
                if not isinstance(state, str):
                    raise TypeError("Each state in the :param states: list must be a string.")
        if not isinstance(name_filter, (str, type(None))):
            raise TypeError(":param name_filter: must be a string or None.")
        if not isinstance(max_clusters, int):
            raise TypeError(":param max_clusters: must be an integer.")
        if max_clusters < 1:
            raise ValueError(":param max_clusters: must be at least 1.")
        # Corrected: Input validation for datetime objects
        if created_after is not None and not isinstance(created_after, datetime):
            raise TypeError(":param created_after: must be a datetime object or None.")
        if created_before is not None and not isinstance(created_before, datetime):
            raise TypeError(":param created_before: must be a datetime object or None.")
        if created_after and created_before and created_after >= created_before:
            raise ValueError(":param created_after: date cannot be on or after :param created_before: date.")



        logger.info("🔍 Discovering EMR clusters in region: %s", self.region)
        logger.info(" States: %s", states)
        logger.info(" Name filter: %s", name_filter or "None")
        logger.info(" Max clusters: %s", max_clusters)
        logger.info(" Created After: %s", created_after or "None")
        logger.info(" Created Before: %s", created_before or "None")


        try:
            # Build parameters for list_clusters
            list_clusters_params = {}
            if states:
                list_clusters_params['ClusterStates'] = states
            else:
                list_clusters_params['ClusterStates'] = ['TERMINATED', 'WAITING'] # Default as per new requirement
            if created_after:
                list_clusters_params['CreatedAfter'] = created_after
            if created_before:
                list_clusters_params['CreatedBefore'] = created_before


            # Use paginator to handle large cluster lists
            paginator = self.emr_client.get_paginator('list_clusters')
            page_iterator = paginator.paginate(**list_clusters_params)



            discovered_clusters = []
            for page in page_iterator:
                for cluster in page.get('Clusters', []):
                    # Apply name filter if specified
                    if name_filter and name_filter.lower() not in cluster.get('Name', '').lower():
                        continue


                    cluster_info = {
                        'cluster_id': cluster.get('Id'),
                        'cluster_name': cluster.get('Name'),
                        'cluster_arn': cluster.get('ClusterArn'),
                        'status': cluster.get('Status', {}).get('State'),
                        'creation_time': cluster.get('Status', {}).get('Timeline', {}).get('CreationDateTime'),
                        'normalized_instance_hours': cluster.get('NormalizedInstanceHours', 0)
                    }
                    discovered_clusters.append(cluster_info)


                    # Stop if we've reached max clusters
                    if len(discovered_clusters) >= max_clusters:
                        break
                if len(discovered_clusters) >= max_clusters:
                    break


            logger.info("✅ Discovered %s clusters", len(discovered_clusters))
            # Log cluster details
            for i, cluster in enumerate(discovered_clusters, 1):
                logger.info(" %d. %s (%s) - %s", i, cluster['cluster_name'], cluster['cluster_id'], cluster['status'])
            return discovered_clusters


        except ClientError as e:
            logger.error("❌ Failed to discover clusters: %s - %s", e.response["Error"]["Code"], e.response["Error"]["Message"], exc_info=True)
            raise
        except Exception as e:
            logger.error("❌ Unexpected error during cluster discovery: %s", str(e), exc_info=True)
            raise


    def get_cluster_details(self, cluster_id: str) -> Dict:
        """
        Get detailed information about a specific cluster.


        :param cluster_id: EMR cluster ID.
        :returns: Detailed cluster information dictionary.
        :raises TypeError: If cluster_id is not a string.
        :raises ClientError: If AWS API call fails.
        """
        if not isinstance(cluster_id, str) or not cluster_id:
            raise TypeError("Cluster ID must be a non-empty string.")
        try:
            response = self.emr_client.describe_cluster(ClusterId=cluster_id)
            cluster = response.get('Cluster', {})


            cluster_details = {
                'cluster_id': cluster.get('Id'),
                'cluster_name': cluster.get('Name'),
                'cluster_arn': cluster.get('ClusterArn'),
                'status': cluster.get('Status', {}).get('State'),
                'emr_release_label': cluster.get('ReleaseLabel'),
                'applications': [app.get('Name') for app in cluster.get('Applications', [])],
                'instance_count': 0,
                'master_instance_type': None,
                'core_instance_type': None,
                'log_uri': cluster.get('LogUri'),
                'ec2_attributes': cluster.get('Ec2InstanceAttributes', {}),
                'creation_time': cluster.get('Status', {}).get('Timeline', {}).get('CreationDateTime'),
                'ready_time': cluster.get('Status', {}).get('Timeline', {}).get('ReadyDateTime'),
                'normalized_instance_hours': cluster.get('NormalizedInstanceHours', 0)
            }


            # Attempt to get instance group information first
            try:
                instance_groups = self.emr_client.list_instance_groups(ClusterId=cluster_id)
                for group in instance_groups.get('InstanceGroups', []):
                    instance_type = group.get('InstanceType')
                    running_count = group.get('RunningInstanceCount', 0)
                    cluster_details['instance_count'] += running_count
                    if group.get('InstanceGroupType') == 'MASTER':
                        cluster_details['master_instance_type'] = instance_type
                    elif group.get('InstanceGroupType') == 'CORE':
                        cluster_details['core_instance_type'] = instance_type
                logger.info("Successfully retrieved instance group info for %s", cluster_id)
            except ClientError as ce:
                # Check if the error is specifically due to instance fleets being used
                if (ce.response["Error"]["Code"] == "InvalidRequestException" and
                        "Instance fleets and instance groups are mutually exclusive" in ce.response["Error"]["Message"]):
                    logger.info("Cluster %s uses instance fleets. Fetching instance fleet details.", cluster_id)
                    # If it uses instance fleets, try to get instance fleet information
                    try:
                        instance_fleets = self.emr_client.list_instance_fleets(ClusterId=cluster_id)
                        for fleet in instance_fleets.get('InstanceFleets', []):
                            # Sum up the target capacities for total instances
                            target_on_demand = fleet.get('TargetOnDemandCapacity', 0)
                            target_spot = fleet.get('TargetSpotCapacity', 0)
                            cluster_details['instance_count'] += target_on_demand + target_spot


                            # Extract master/core instance types from fleet details
                            # InstanceTypeSpecifications is a LIST, not a dict
                            instance_type_specs = fleet.get('InstanceTypeSpecifications', [])
                            if fleet.get('InstanceFleetType') == 'MASTER' and instance_type_specs:
                                cluster_details['master_instance_type'] = instance_type_specs[0].get('InstanceType')
                            elif fleet.get('InstanceFleetType') == 'CORE' and instance_type_specs:
                                cluster_details['core_instance_type'] = instance_type_specs[0].get('InstanceType')
                        logger.info("Successfully retrieved instance fleet info for %s", cluster_id)
                    except ClientError as fleet_ce:
                        logger.warning("Could not get instance fleet details for %s: %s - %s",
                                       cluster_id, fleet_ce.response["Error"]["Code"],
                                       fleet_ce.response["Error"]["Message"], exc_info=True)
                    except Exception as fleet_e:
                        logger.warning("Unexpected error getting instance fleet details for %s: %s",
                                       cluster_id, str(fleet_e), exc_info=True)
                else:
                    # Re-raise any other ClientErrors that are not the mutual exclusivity error
                    logger.warning("Could not get instance group details for %s: %s - %s",
                                   cluster_id, ce.response["Error"]["Code"],
                                   ce.response["Error"]["Message"], exc_info=True)
                    raise # Re-raise original ClientError
            except Exception as e:
                # Catch any other general exceptions during initial instance group/fleet fetching
                logger.warning("Unexpected error during instance details fetching for %s: %s",
                               cluster_id, str(e), exc_info=True)


            return cluster_details


        except ClientError as e:
            logger.error("❌ Failed to describe cluster %s: %s - %s",
                         cluster_id, e.response["Error"]["Code"],
                         e.response["Error"]["Message"], exc_info=True)
            raise
        except Exception as e:
            logger.error("❌ Unexpected error getting cluster details for %s: %s",
                         cluster_id, str(e), exc_info=True)
            raise


    def validate_cluster_for_analysis(self, cluster_info: Dict) -> bool:
        """
        Validate if a cluster is suitable for Spark History Server analysis.
        A cluster is suitable if it has the Spark application installed and is in a state
        where history data might be available.


        :param cluster_info: Cluster information dictionary.
        :returns: True if cluster is suitable for analysis, False otherwise.
        :raises TypeError: If cluster_info is not a dictionary or has incorrect types.
        """
        if not isinstance(cluster_info, dict):
            raise TypeError(":param cluster_info: must be a dictionary.")


        cluster_id = cluster_info.get('cluster_id')
        status = cluster_info.get('status')
        applications = cluster_info.get('applications', [])


        if not isinstance(cluster_id, str):
            raise TypeError("Cluster ID in cluster_info must be a string.")
        if not isinstance(status, str):
            raise TypeError("Status in cluster_info must be a string.")
        if not isinstance(applications, list):
            raise TypeError("Applications in cluster_info must be a list.")
        for app in applications:
            if not isinstance(app, str):
                raise TypeError("Each application name in cluster_info must be a string.")


        # Check if cluster has Spark application
        has_spark = any('Spark' in app for app in applications)
        if not has_spark:
            logger.warning("⚠️ Cluster %s does not have Spark installed - skipping", cluster_id)
            return False


        # Check if cluster is in a valid state for analysis.
        # Even 'TERMINATED' clusters can have history data.
        valid_states = ['RUNNING', 'WAITING', 'TERMINATED']
        if status not in valid_states:
            logger.warning("⚠️ Cluster %s is in state '%s' - may not have history data or is not a target state for analysis", cluster_id, status)
            return False # Exclude clusters not in RUNNING, WAITING, or TERMINATED


        logger.info("✅ Cluster %s is valid for analysis (Spark: %s, State: %s)", cluster_id, has_spark, status)
        return True


### Spark History Server REST Interaction

In [0]:
import json
from datetime import datetime
from typing import List, Any, Dict, Optional

import requests
import logging

logger = logging.getLogger(__name__)

class SparkHistoryServerClient:
    """Client for interacting with Spark History Server REST API."""

    def __init__(self, base_url: str, session: requests.Session) -> None:
        """
        Initialize the Spark History Server client.

        :param base_url: Base URL for the Spark History Server
        :param session: Configured HTTP session with authentication
        """
        self.base_url = base_url
        self.session = session
        self.api_base = f"{base_url}/api/v1"

    def _make_request(self, endpoint: str, params: Optional[Dict] = None) -> Any:
        """
        Make a REST API request to the Spark History Server.

        :param endpoint: API endpoint (relative to /api/v1)
        :param params: Query parameters
        :returns: JSON response
        :raises requests.exceptions.RequestException: If request fails
        :raises json.JSONDecodeError: If JSON parsing fails
        """
        url = f"{self.api_base}/{endpoint}"
        try:
            response = self.session.get(url, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            logger.error("❌ Failed to make request to %s: %s", url, str(e), exc_info=True)
            raise
        except json.JSONDecodeError as e:
            logger.error("❌ Failed to parse JSON response from %s: %s", url, str(e), exc_info=True)
            raise

    def get_applications(self, status: Optional[str] = None, limit: int = 100) -> List[Dict]:
        """
        Get list of Spark applications.

        :param status: Filter by status (running, completed, failed)
        :param limit: Maximum number of applications to return
        :returns: List of application metadata
        """
        logger.info("Fetching applications (status: %s, limit: %s)", status, limit)
        params = {}
        if status:
            params['status'] = status
        if limit:
            params['limit'] = limit
        applications = self._make_request("applications", params)
        logger.info("✅ Retrieved %s applications", len(applications))
        return applications

    def get_application_details(self, app_id: str) -> Dict:
        """
        Get detailed information about a specific application.

        :param app_id: Application ID
        :returns: Application details
        """
        logger.info("Fetching application details for: %s", app_id)
        details = self._make_request(f"applications/{app_id}")
        logger.info("✅ Retrieved details for application: %s", app_id)
        return details

    def get_application_jobs(self, app_id: str, attempt_id: Optional[str] = None) -> List[Dict]:
        """
        Get jobs for a specific application.

        :param app_id: Application ID
        :param attempt_id: The application attempt ID. If None, uses the base application endpoint.
        :returns: List of jobs
        """
        logger.info("Fetching jobs for application: %s (Attempt ID: %s)", app_id, attempt_id or "N/A")
        endpoint = f"applications/{app_id}"
        if attempt_id:
            endpoint += f"/{attempt_id}"
        jobs = self._make_request(f"{endpoint}/jobs")
        logger.info("✅ Retrieved %s jobs for application: %s", len(jobs), app_id)
        return jobs

    def get_application_stages(self, app_id: str, attempt_id: Optional[str] = None) -> List[Dict]:
        """
        Get stages for a specific application.

        :param app_id: Application ID
        :param attempt_id: The application attempt ID. If None, uses the base application endpoint.
        :returns: List of stages
        """
        logger.info("Fetching stages for application: %s (Attempt ID: %s)", app_id, attempt_id or "N/A")
        endpoint = f"applications/{app_id}"
        if attempt_id:
            endpoint += f"/{attempt_id}"
        stages = self._make_request(f"{endpoint}/stages")
        logger.info("✅ Retrieved %s stages for application: %s", len(stages), app_id)
        return stages

    def get_application_executors(self, app_id: str) -> List[Dict]:
        """
        Get executors for a specific application.

        :param app_id: Application ID
        :returns: List of executors
        """
        logger.info("Fetching executors for application: %s", app_id)
        executors = self._make_request(f"applications/{app_id}/executors")
        logger.info("✅ Retrieved %s executors for application: %s", len(executors), app_id)
        return executors

    def get_stage_tasks(self, app_id: str, stage_id: int, stage_attempt: int = 0) -> List[Dict]:
        """
        Get tasks for a specific stage.

        :param app_id: Application ID
        :param stage_id: Stage ID
        :param stage_attempt: Stage attempt number
        :returns: List of tasks
        """
        logger.info("Fetching tasks for stage %s (attempt %s) in application: %s", stage_id, stage_attempt, app_id)
        tasks = self._make_request(f"applications/{app_id}/stages/{stage_id}/{stage_attempt}/taskList")
        logger.info("✅ Retrieved %s tasks for stage %s", len(tasks), stage_id)
        return tasks

    def get_application_sql_queries(self, app_id: str, min_duration_minutes: int = 0, attempt_id: Optional[str] = None) -> List[Dict]:
        """
        Get SQL queries for a specific application and filter by minimum duration.

        :param app_id: Application ID
        :param min_duration_minutes: Minimum duration in minutes for SQL queries to include
        :param attempt_id: The application attempt ID. If None, uses the base application endpoint.
        :returns: List of SQL query data (raw JSON)
        """
        logger.info("Fetching SQL queries for application: %s (min_duration_minutes: %s, Attempt ID: %s)", app_id, min_duration_minutes, attempt_id or "N/A")
        endpoint = f"applications/{app_id}"
        if attempt_id:
            endpoint += f"/{attempt_id}"
        sql_queries = self._make_request(f"{endpoint}/sql")
        logger.info("✅ Retrieved %s raw SQL queries for application: %s", len(sql_queries), app_id)

        # Filter by duration and include the full JSON
        filtered_queries = []
        for query in sql_queries:
            duration_ms = query.get("duration", 0)
            if duration_ms >= (min_duration_minutes * 60 * 1000):
                filtered_queries.append(query)
        
        logger.info("✅ Filtered to %s SQL queries with duration >= %s minutes for application: %s", len(filtered_queries), min_duration_minutes, app_id)
        return filtered_queries


### Metrics Analysis Functions

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, ArrayType, MapType, IntegerType
from typing import List, Dict, Tuple
import pandas as pd
import logging
import json

logger = logging.getLogger(__name__)

class SparkMetricsAnalyzer:
    """Analyzer for Spark application metrics and performance data."""

    def __init__(self, spark: SparkSession) -> None:
        """
        Initialize the metrics analyzer.
        :param spark: Spark session
        """
        self.spark = spark

    def analyze_application_performance(self, app_data: Dict) -> Dict:
        """
        Analyze performance metrics for a single application.
        :param app_data: Application data from Spark History Server
        :returns: Performance analysis results
        """
        duration_ms = app_data.get('duration') or 0
        
        analysis = {
            'application_id': app_data.get('id'),
            'application_name': app_data.get('name') or 'Unknown',
            'duration_ms': duration_ms,
            'duration_minutes': 0.0, # Placeholder, will be calculated next
            'start_time': app_data.get('startTime'),
            'end_time': app_data.get('endTime'),
            'spark_version': app_data.get('sparkVersion') or 'Unknown',
            'cores': app_data.get('cores') or 0,
            'memory_per_executor_mb': app_data.get('memoryPerExecutorMB') or 0,
            'max_cores': app_data.get('maxCores') or 0,
            'core_hours': 0.0,
            'efficiency_score': 0.0,
            'cluster_id': '', # Will be populated by caller
            'cluster_name': '' # Will be populated by caller
        }

        # Explicitly use Python's built-in round function to avoid name conflicts
        analysis['duration_minutes'] = __builtins__.round(duration_ms / (1000 * 60), 2)

        # Calculate efficiency metrics - ensure no None values
        cores = analysis['cores']
        max_cores = analysis['max_cores']
        if duration_ms > 0 and cores > 0:
            analysis['core_hours'] = (duration_ms / (1000 * 3600)) * cores
            analysis['efficiency_score'] = min(100.0, (cores / max(1, max_cores)) * 100.0)

        return analysis

    def analyze_job_performance(self, jobs: List[Dict]) -> List[Dict]:
        """
        Analyze performance metrics for jobs.
        :param jobs: List of job data
        :returns: Job performance analysis
        """
        job_analysis = []
        for job in jobs:
            analysis = {
                'job_id': job.get('jobId'),
                'job_name': job.get('name') or 'Unknown',
                'status': job.get('status') or 'UNKNOWN',
                'submission_time': job.get('submissionTime'),
                'completion_time': job.get('completionTime'),
                'num_tasks': job.get('numTasks') or 0,
                'num_active_tasks': job.get('numActiveTasks') or 0,
                'num_completed_tasks': job.get('numCompletedTasks') or 0,
                'num_failed_tasks': job.get('numFailedTasks') or 0,
                'num_stages': len(job.get('stageIds', [])),
                'stage_ids': str(job.get('stageIds', [])), # Convert to string for schema consistency
                'task_success_rate': 0.0,
                'cluster_id': '', # Will be populated by caller
                'cluster_name': '', # Will be populated by caller
                'application_id': '' # Will be populated by caller
            }

            # Calculate success rate - ensure no None values
            num_tasks = analysis['num_tasks']
            num_completed = analysis['num_completed_tasks']
            if num_tasks > 0:
                analysis['task_success_rate'] = (num_completed / num_tasks) * 100.0
            
            job_analysis.append(analysis)
        return job_analysis

    def analyze_stage_performance(self, stages: List[Dict]) -> List[Dict]:
        """
        Analyze performance metrics for stages.
        :param stages: List of stage data
        :returns: Stage performance analysis
        """
        stage_analysis = []
        for stage in stages:
            analysis = {
                'stage_id': stage.get('stageId'),
                'stage_name': stage.get('name') or 'Unknown',
                'status': stage.get('status') or 'UNKNOWN',
                'num_tasks': stage.get('numTasks') or 0,
                'num_active_tasks': stage.get('numActiveTasks') or 0,
                'num_complete_tasks': stage.get('numCompleteTasks') or 0,
                'num_failed_tasks': stage.get('numFailedTasks') or 0,
                'executor_run_time': stage.get('executorRunTime') or 0,
                'executor_cpu_time': stage.get('executorCpuTime') or 0,
                'submission_time': stage.get('submissionTime'),
                'first_task_launched_time': stage.get('firstTaskLaunchedTime'),
                'completion_time': stage.get('completionTime'),
                'input_bytes': stage.get('inputBytes') or 0,
                'output_bytes': stage.get('outputBytes') or 0,
                'shuffle_read_bytes': stage.get('shuffleReadBytes') or 0,
                'shuffle_write_bytes': stage.get('shuffleWriteBytes') or 0,
                'memory_bytes_spilled': stage.get('memoryBytesSpilled') or 0,
                'disk_bytes_spilled': stage.get('diskBytesSpilled') or 0,
                'task_completion_rate': 0.0,
                'avg_executor_run_time_per_task': 0.0,
                'total_data_processed_mb': 0.0,
                'shuffle_data_mb': 0.0,
                'cluster_id': '', # Will be populated by caller
                'cluster_name': '', # Will be populated by caller
                'application_id': '' # Will be populated by caller
            }

            # Calculate efficiency metrics - ensure no None values
            num_tasks = analysis['num_tasks']
            num_complete = analysis['num_complete_tasks']
            executor_run_time = analysis['executor_run_time']
            input_bytes = analysis['input_bytes']
            output_bytes = analysis['output_bytes']
            shuffle_read_bytes = analysis['shuffle_read_bytes']
            shuffle_write_bytes = analysis['shuffle_write_bytes']

            if num_tasks > 0:
                analysis['task_completion_rate'] = (num_complete / num_tasks) * 100.0
                analysis['avg_executor_run_time_per_task'] = executor_run_time / num_tasks

            # Data processing metrics
            analysis['total_data_processed_mb'] = (input_bytes + output_bytes) / (1024 * 1024)
            analysis['shuffle_data_mb'] = (shuffle_read_bytes + shuffle_write_bytes) / (1024 * 1024)
            
            stage_analysis.append(analysis)
        return stage_analysis

    def analyze_sql_queries(self, sql_queries: List[Dict]) -> List[Dict]:
        """
        Analyze SQL query metrics.
        Extracts key information and stores the full JSON as a string.
        The `description` and `nodes` fields from the JSON are preserved.
        :param sql_queries: List of raw SQL query data from Spark History Server.
        :returns: List of processed SQL query analysis dictionaries.
        """
        sql_analysis = []
        for query in sql_queries:
            # Safely get description and nodes, defaulting to empty or sensible values
            description = query.get("description", "N/A")
            nodes = query.get("nodes", [])

            # Extract metrics from nodes for summary.
            node_metrics = {}
            if nodes and isinstance(nodes, list) and len(nodes) > 0:
                first_node = nodes[0]
                if "metrics" in first_node and isinstance(first_node["metrics"], list):
                    for metric in first_node["metrics"]:
                        if "name" in metric and "value" in metric:
                            metric_name = metric["name"].replace(" ", "_").replace(".", "_").lower()
                            node_metrics[metric_name] = str(metric["value"]) # Store as string for flexibility

            duration_ms = query.get("duration", 0)
            duration_minutes = __builtins__.round(duration_ms / (1000 * 60), 2)

            analysis = {
                'sql_id': query.get("id"),
                'description': description,
                'status': query.get("status"),
                'duration_ms': duration_ms,
                'duration_minutes': duration_minutes,
                'submission_time': query.get("submissionTime"),
                'success_job_ids': str(query.get("successJobIds", [])),
                'failed_job_ids': str(query.get("failedJobIds", [])),
                'num_nodes': len(nodes) if nodes else 0,
                'sql_raw_json': json.dumps(query), # Store full JSON as a string
                'cluster_id': '', # Will be populated by caller
                'cluster_name': '', # Will be populated by caller
                'application_id': '' # Will be populated by caller
            }
            # Add extracted node metrics dynamically
            analysis.update(node_metrics)
            sql_analysis.append(analysis)
        
        return sql_analysis

    def create_performance_dataframes(
        self,
        applications_analysis: List[Dict],
        jobs_analysis: List[Dict],
        stages_analysis: List[Dict],
        sql_analysis: List[Dict]
    ) -> Tuple[Optional[Any], Optional[Any], Optional[Any], Optional[Any]]:
        """
        Create Spark DataFrames from analysis results with explicit schemas.
        :param applications_analysis: Application analysis results
        :param jobs_analysis: Job analysis results
        :param stages_analysis: Stage analysis results
        :param sql_analysis: SQL query analysis results
        :returns: Tuple of (applications_df, jobs_df, stages_df, sql_df)
        """
        # Define explicit schemas
        applications_schema = StructType([
            StructField("cluster_id", StringType(), True),
            StructField("cluster_name", StringType(), True),
            StructField("application_id", StringType(), True),
            StructField("application_name", StringType(), True),
            StructField("duration_ms", LongType(), True),
            StructField("duration_minutes", DoubleType(), True),
            StructField("start_time", StringType(), True),
            StructField("end_time", StringType(), True),
            StructField("spark_version", StringType(), True),
            StructField("cores", LongType(), True),
            StructField("memory_per_executor_mb", LongType(), True),
            StructField("max_cores", LongType(), True),
            StructField("core_hours", DoubleType(), True),
            StructField("efficiency_score", DoubleType(), True)
        ])
        jobs_schema = StructType([
            StructField("cluster_id", StringType(), True),
            StructField("cluster_name", StringType(), True),
            StructField("application_id", StringType(), True),
            StructField("job_id", LongType(), True),
            StructField("job_name", StringType(), True),
            StructField("status", StringType(), True),
            StructField("submission_time", StringType(), True),
            StructField("completion_time", StringType(), True),
            StructField("num_tasks", LongType(), True),
            StructField("num_active_tasks", LongType(), True),
            StructField("num_completed_tasks", LongType(), True),
            StructField("num_failed_tasks", LongType(), True),
            StructField("num_stages", LongType(), True),
            StructField("stage_ids", StringType(), True),
            StructField("task_success_rate", DoubleType(), True)
        ])
        stages_schema = StructType([
            StructField("cluster_id", StringType(), True),
            StructField("cluster_name", StringType(), True),
            StructField("application_id", StringType(), True),
            StructField("stage_id", LongType(), True),
            StructField("stage_name", StringType(), True),
            StructField("status", StringType(), True),
            StructField("num_tasks", LongType(), True),
            StructField("num_active_tasks", LongType(), True),
            StructField("num_complete_tasks", LongType(), True),
            StructField("num_failed_tasks", LongType(), True),
            StructField("executor_run_time", LongType(), True),
            StructField("executor_cpu_time", LongType(), True),
            StructField("submission_time", StringType(), True),
            StructField("first_task_launched_time", StringType(), True),
            StructField("completion_time", StringType(), True),
            StructField("input_bytes", LongType(), True),
            StructField("output_bytes", LongType(), True),
            StructField("shuffle_read_bytes", LongType(), True),
            StructField("shuffle_write_bytes", LongType(), True),
            StructField("memory_bytes_spilled", LongType(), True),
            StructField("disk_bytes_spilled", LongType(), True),
            StructField("task_completion_rate", DoubleType(), True),
            StructField("avg_executor_run_time_per_task", DoubleType(), True),
            StructField("total_data_processed_mb", DoubleType(), True),
            StructField("shuffle_data_mb", DoubleType(), True)
        ])
        sql_schema = StructType([
            StructField("cluster_id", StringType(), True),
            StructField("cluster_name", StringType(), True),
            StructField("application_id", StringType(), True),
            StructField("sql_id", LongType(), True),
            StructField("description", StringType(), True),
            StructField("status", StringType(), True),
            StructField("duration_ms", LongType(), True),
            StructField("duration_minutes", DoubleType(), True),
            StructField("submission_time", StringType(), True),
            StructField("success_job_ids", StringType(), True),
            StructField("failed_job_ids", StringType(), True),
            StructField("num_nodes", LongType(), True),
            StructField("sql_raw_json", StringType(), True)
        ])

        apps_df = self.spark.createDataFrame(applications_analysis, schema=applications_schema) if applications_analysis else None
        jobs_df = self.spark.createDataFrame(jobs_analysis, schema=jobs_schema) if jobs_analysis else None
        stages_df = self.spark.createDataFrame(stages_analysis, schema=stages_schema) if stages_analysis else None
        sql_df = self.spark.createDataFrame(sql_analysis, schema=sql_schema) if sql_analysis else None

        return apps_df, jobs_df, stages_df, sql_df


### Cluster Analyzer


In [0]:
import concurrent.futures
from typing import Dict, List, Any, Optional, Tuple
import logging

logger = logging.getLogger(__name__)

def analyze_single_cluster(
    cluster_info: Dict,
    shs_client_class: Any,
    analyzer_class: Any,
    timeout_seconds: int,
    max_applications: int,
    spark_session: 'SparkSession',
    persistent_ui_timeout: int,
    max_app_threads: int
) -> Dict[str, Any]:
    """
    Analyzes a single EMR cluster, fetching application, job, stage, and SQL query data.
    This function is designed to be executed concurrently.

    :param cluster_info: Dictionary containing cluster details (id, name, arn, status).
    :param shs_client_class: The SparkHistoryServerClient class to instantiate.
    :param analyzer_class: The SparkMetricsAnalyzer class to instantiate.
    :param timeout_seconds: Timeout for HTTP requests to the Spark History Server.
    :param max_applications: Maximum number of applications to analyze per cluster.
    :param spark_session: The active SparkSession for creating DataFrames.
    :param persistent_ui_timeout: The maximum time in seconds to wait for the Persistent App UI.
    :param max_app_threads: The maximum number of concurrent threads for analyzing applications.
    :returns: A dictionary containing analyzed data and a summary for the cluster.
    """
    cluster_id = cluster_info['cluster_id']
    cluster_name = cluster_info['cluster_name']
    cluster_arn = cluster_info['cluster_arn']

    logger.info("🕵️ Starting analysis for cluster: %s (%s)", cluster_name, cluster_id)

    cluster_analysis_results = {
        'cluster_id': cluster_id,
        'cluster_name': cluster_name,
        'status': cluster_info.get('status', 'UNKNOWN'),
        'total_applications': 0,
        'total_jobs': 0,
        'total_stages': 0,
        'total_sql_queries': 0,
        'analysis_status': 'FAILED',
        'applications': [],
        'jobs': [],
        'stages': [],
        'sql_queries': [],
        'applications_endpoint_success': None,
        'jobs_endpoint_success': None,
        'sql_endpoint_success': None,
        'error_message': ""
    }

    try:
        server_config = ServerConfig(
            emr_cluster_arn=cluster_arn,
            timeout=timeout_seconds
        )
        emr_client = EMRPersistentUIClient(server_config)

        logger.info("🔑 Initializing EMR Persistent UI connection for %s...", cluster_name)
        base_url, session = emr_client.initialize(max_wait_time=persistent_ui_timeout)

        logger.info("📊 Setting up Spark History Server client for %s...", cluster_name)
        shs_client = shs_client_class(base_url, session)
        analyzer = analyzer_class(spark_session)

        logger.info("📄 Fetching Spark applications for %s...", cluster_name)
        try:
            applications = shs_client.get_applications(limit=max_applications)
            cluster_analysis_results['applications_endpoint_success'] = True
        except Exception as app_ex:
            cluster_analysis_results['applications_endpoint_success'] = False
            cluster_analysis_results['error_message'] += f"Applications endpoint error: {str(app_ex)}; "
            logger.error("❌ Failed to fetch applications for %s: %s", cluster_name, str(app_ex), exc_info=True)
            return cluster_analysis_results

        if not applications:
            logger.warning("⚠️ No applications found in Spark History Server for %s", cluster_name)
            cluster_analysis_results['analysis_status'] = 'NO_APPLICATIONS'
            return cluster_analysis_results

        logger.info("✅ Found %s applications to analyze in %s", len(applications), cluster_name)

        app_futures = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_app_threads) as app_executor:
            for app in applications:
                app_id = app.get('id')
                app_futures.append(app_executor.submit(
                    process_single_application,
                    app_id,
                    shs_client,
                    analyzer,
                    cluster_analysis_results
                ))

        for future in concurrent.futures.as_completed(app_futures):
            try:
                app_data = future.result()
                if app_data:
                    cluster_analysis_results['applications'].append(app_data['application'])
                    cluster_analysis_results['jobs'].extend(app_data['jobs'])
                    cluster_analysis_results['stages'].extend(app_data['stages'])
                    cluster_analysis_results['sql_queries'].extend(app_data['sql_queries'])
                    cluster_analysis_results['total_applications'] += 1
                    cluster_analysis_results['total_jobs'] += len(app_data['jobs'])
                    cluster_analysis_results['total_stages'] += len(app_data['stages'])
                    cluster_analysis_results['total_sql_queries'] += len(app_data['sql_queries'])
            except Exception as app_e:
                logger.error(" ❌ Error analyzing an application in cluster %s: %s", cluster_name, str(app_e), exc_info=True)

        cluster_analysis_results['analysis_status'] = 'COMPLETED'
        logger.info("✅ Completed analysis for cluster: %s", cluster_name)
        logger.info(
            " Applications: %s, Jobs: %s, Stages: %s, SQL Queries: %s",
            cluster_analysis_results['total_applications'],
            cluster_analysis_results['total_jobs'],
            cluster_analysis_results['total_stages'],
            cluster_analysis_results['total_sql_queries']
        )
        return cluster_analysis_results

    except Exception as e:
        logger.error("❌ Failed to analyze cluster %s: %s", cluster_name, str(e), exc_info=True)
        cluster_analysis_results['analysis_status'] = 'FAILED'
        cluster_analysis_results['error_message'] += f"Cluster analysis error: {str(e)}"
        return cluster_analysis_results

def process_single_application(app_id: str, shs_client: Any, analyzer: Any, cluster_results: Dict) -> Optional[Dict[str, Any]]:
    """
    Processes a single Spark application's data. Designed for concurrent execution.
    """
    logger.info(" 🕵️ Analyzing application: %s", app_id)
    app_results = {'application': None, 'jobs': [], 'stages': [], 'sql_queries': []}
    try:
        app_details = shs_client.get_application_details(app_id)

        latest_attempt_id_to_use = None
        attempts = app_details.get('attempts')
        if attempts and isinstance(attempts, list):
            attempt_ids = [int(a['attemptId']) for a in attempts if a.get('attemptId')]
            if attempt_ids:
                latest_attempt_id = max(attempt_ids)
                logger.info(" Latest attempt ID for application %s is: %d", app_id, latest_attempt_id)
                if latest_attempt_id > 1:
                    latest_attempt_id_to_use = str(latest_attempt_id)
                    logger.info(" Will use attempt ID %s for API calls.", latest_attempt_id_to_use)
                else:
                    logger.info(" Latest attempt ID is 1, so it will not be appended to the URL.")

        app_results['application'] = analyzer.analyze_application_performance(app_details)

        try:
            jobs = shs_client.get_application_jobs(app_id, attempt_id=latest_attempt_id_to_use)
            if jobs:
                app_results['jobs'] = analyzer.analyze_job_performance(jobs)
            if cluster_results['jobs_endpoint_success'] is None:
                cluster_results['jobs_endpoint_success'] = True
        except Exception as job_ex:
            if cluster_results['jobs_endpoint_success'] is None:
                cluster_results['jobs_endpoint_success'] = False
            cluster_results['error_message'] += f"Jobs endpoint error: {str(job_ex)}; "
            logger.error(" ❌ Failed to fetch jobs for application %s: %s", app_id, str(job_ex), exc_info=True)

        try:
            stages = shs_client.get_application_stages(app_id, attempt_id=latest_attempt_id_to_use)
            if stages:
                app_results['stages'] = analyzer.analyze_stage_performance(stages)
        except Exception as stage_ex:
            logger.error(" ❌ Failed to fetch stages for application %s: %s", app_id, str(stage_ex), exc_info=True)

        try:
            sql_queries = shs_client.get_application_sql_queries(app_id, attempt_id=latest_attempt_id_to_use)
            if sql_queries:
                app_results['sql_queries'] = analyzer.analyze_sql_queries(sql_queries)
            if cluster_results['sql_endpoint_success'] is None:
                cluster_results['sql_endpoint_success'] = True
        except Exception as sql_ex:
            if cluster_results['sql_endpoint_success'] is None:
                cluster_results['sql_endpoint_success'] = False
            cluster_results['error_message'] += f"SQL endpoint error: {str(sql_ex)}; "
            logger.error(" ❌ Failed to fetch SQL queries for application %s: %s", app_id, str(sql_ex), exc_info=True)

        return app_results
    except Exception as e:
        logger.error(" ❌ Failed to analyze application %s: %s", app_id, str(e), exc_info=True)
        return None

def summarize_cluster_status(cluster_results: Dict) -> str:
    """
    Generate status details based on cluster analysis results.
    """
    status_details = ""

    if cluster_results['analysis_status'] == 'FAILED':
        status_details = cluster_results.get('error_message', 'Unknown error - see logs')
    elif cluster_results['analysis_status'] == 'COMPLETED':
        endpoint_statuses = []
        if cluster_results.get('applications_endpoint_success') is not None:
            endpoint_statuses.append(f"applications: {'OK' if cluster_results['applications_endpoint_success'] else 'FAILED'}")
        if cluster_results.get('jobs_endpoint_success') is not None:
            endpoint_statuses.append(f"jobs: {'OK' if cluster_results['jobs_endpoint_success'] else 'FAILED'}")
        if cluster_results.get('sql_endpoint_success') is not None:
            endpoint_statuses.append(f"sql: {'OK' if cluster_results['sql_endpoint_success'] else 'FAILED'}")
        status_details = "; ".join(endpoint_statuses) if endpoint_statuses else "Endpoints not checked"
    elif cluster_results['analysis_status'] == 'NO_APPLICATIONS':
        status_details = "No applications found in Spark History Server"

    return status_details


### Main Execution

In [0]:
import concurrent.futures
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from typing import Dict, Any
import logging

# This code assumes that all necessary classes (e.g., EMRClusterDiscovery), 
# configuration variables (e.g., EMR_CLUSTER_ARN), and the functions 
# from the 'Cluster Analyzer' block are defined in preceding cells.

logger = logging.getLogger(__name__)

def main_analysis() -> Dict[str, Any]:
    """
    Main function to execute the complete Spark History Server analysis.
    Supports both single cluster analysis and multi-cluster discovery.
    Returns all data as DataFrames without creating temp views.

    :returns: Dictionary containing all analysis DataFrames and summaries
    :raises Exception: If any step of the analysis fails
    """
    try:
        logger.info("🚀 Starting EMR Spark History Server Analysis")

        clusters_to_analyze = []
        if EMR_CLUSTER_ARN:
            logger.info("🎯 Single cluster mode - using provided EMR Cluster ARN")
            cluster_id = EMR_CLUSTER_ARN.split('/')[-1]
            clusters_to_analyze.append({'cluster_id': cluster_id, 'cluster_name': 'User-Specified', 'cluster_arn': EMR_CLUSTER_ARN, 'status': 'UNKNOWN'})
        else:
            logger.info("🔭 Multi-cluster discovery mode - searching for EMR clusters")
            discovery = EMRClusterDiscovery(AWS_REGION)
            discovered_clusters = discovery.discover_clusters(
                states=CLUSTER_STATES_LIST,
                name_filter=CLUSTER_NAME_FILTER,
                max_clusters=MAX_CLUSTERS,
                created_after=PARSED_CREATED_AFTER_DATE,
                created_before=PARSED_CREATED_BEFORE_DATE
            )
            if not discovered_clusters:
                logger.warning("⚠️ No clusters found matching criteria")
                return {'cluster_summaries_df': None, 'applications_df': None, 'jobs_df': None, 'stages_df': None, 'sql_df': None, 'summary': {'total_clusters_analyzed': 0, 'total_applications': 0, 'total_jobs': 0, 'total_stages': 0, 'total_sql_queries': 0}}

            for cluster_summary in discovered_clusters:
                try:
                    cluster_details = discovery.get_cluster_details(cluster_summary['cluster_id'])
                    if discovery.validate_cluster_for_analysis(cluster_details):
                        clusters_to_analyze.append({'cluster_id': cluster_details['cluster_id'], 'cluster_name': cluster_details['cluster_name'], 'cluster_arn': cluster_details['cluster_arn'], 'status': cluster_details['status'], 'details': cluster_details})
                except Exception as e:
                    logger.error("❌ Failed to validate cluster %s: %s", cluster_summary['cluster_id'], str(e), exc_info=True)
                    continue

        if not clusters_to_analyze:
            logger.error("❌ No valid clusters found for analysis")
            return {'cluster_summaries_df': None, 'applications_df': None, 'jobs_df': None, 'stages_df': None, 'sql_df': None, 'summary': {'total_clusters_analyzed': 0, 'total_applications': 0, 'total_jobs': 0, 'total_stages': 0, 'total_sql_queries': 0}}

        logger.info("📊 Will analyze %s cluster(s)", len(clusters_to_analyze))

        spark = SparkSession.builder.appName("EMR-Multi-Cluster-Spark-History-Analysis").getOrCreate()
        analyzer_instance = SparkMetricsAnalyzer(spark)

        all_applications_analysis, all_jobs_analysis, all_stages_analysis, all_sql_analysis, cluster_summaries = [], [], [], [], []

        with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_CLUSTER_THREADS) as cluster_executor:
            cluster_futures = {
                cluster_executor.submit(
                    analyze_single_cluster,
                    c_info,
                    SparkHistoryServerClient,
                    SparkMetricsAnalyzer,
                    TIMEOUT_SECONDS,
                    MAX_APPLICATIONS,
                    spark,
                    PERSISTENT_UI_TIMEOUT_SECONDS,
                    MAX_APP_THREADS
                ): c_info['cluster_id']
                for c_info in clusters_to_analyze
            }

            for future in concurrent.futures.as_completed(cluster_futures):
                cluster_id = cluster_futures[future]
                try:
                    cluster_results = future.result()
                    if cluster_results:
                        cluster_summaries.append({'cluster_id': cluster_results['cluster_id'], 'cluster_name': cluster_results['cluster_name'], 'status': cluster_results['status'], 'analysis_status': cluster_results['analysis_status'], 'status_details': summarize_cluster_status(cluster_results), 'total_applications': cluster_results['total_applications'], 'total_jobs': cluster_results['total_jobs'], 'total_stages': cluster_results['total_stages'], 'total_sql_queries': cluster_results['total_sql_queries']})
                        for item_list, key in [(all_applications_analysis, 'applications'), (all_jobs_analysis, 'jobs'), (all_stages_analysis, 'stages'), (all_sql_analysis, 'sql_queries')]:
                            for data in cluster_results[key]:
                                data['cluster_id'] = cluster_results['cluster_id']
                                data['cluster_name'] = cluster_results['cluster_name']
                                item_list.append(data)
                except Exception as e:
                    logger.error("❌ Error processing results for cluster %s: %s", cluster_id, str(e), exc_info=True)
                    if not any(s['cluster_id'] == cluster_id for s in cluster_summaries):
                        cluster_name = next((c['cluster_name'] for c in clusters_to_analyze if c['cluster_id'] == cluster_id), 'Unknown')
                        cluster_summaries.append({'cluster_id': cluster_id, 'cluster_name': cluster_name, 'status': 'FAILED', 'total_applications': 0, 'total_jobs': 0, 'total_stages': 0, 'total_sql_queries': 0, 'analysis_status': 'FAILED', 'status_details': f"Error processing cluster results: {str(e)}"})

        logger.info("💾 Creating analysis DataFrames...")
        apps_df, jobs_df, stages_df, sql_df = analyzer_instance.create_performance_dataframes(all_applications_analysis, all_jobs_analysis, all_stages_analysis, all_sql_analysis)

        cluster_summary_df = None
        if cluster_summaries:
            cluster_summary_schema = StructType([StructField('cluster_id', StringType(), True), StructField('cluster_name', StringType(), True), StructField('status', StringType(), True), StructField('analysis_status', StringType(), True), StructField('status_details', StringType(), True), StructField('total_applications', IntegerType(), True), StructField('total_jobs', IntegerType(), True), StructField('total_stages', IntegerType(), True), StructField('total_sql_queries', IntegerType(), True)])
            cluster_summary_df = spark.createDataFrame(cluster_summaries, schema=cluster_summary_schema)

        logger.info("📝 Multi-Cluster Analysis Complete - Summary Results:")
        print("\n" + "="*100 + "\n📊 CLUSTER ANALYSIS SUMMARY\n" + "="*100)
        if cluster_summary_df:
            print(f"Cluster Summary ({cluster_summary_df.count()} clusters):")
            cluster_summary_df.show(truncate=False)

        if apps_df and apps_df.count() > 0:
            logger.info(" • Total Applications analyzed: %s", apps_df.count())
            print("\n=== TOP 10 APPLICATIONS BY DURATION (ACROSS ALL CLUSTERS) ===")
            apps_df.select("cluster_name", "application_name", "duration_minutes", "cores", "efficiency_score").orderBy("duration_minutes", ascending=False).show(10, truncate=False)

        if jobs_df and jobs_df.count() > 0:
            logger.info(" • Total Jobs analyzed: %s", jobs_df.count())
            print("\n=== JOB SUCCESS RATES BY CLUSTER ===")
            jobs_df.select("cluster_name", "application_id", "job_name", "status", "task_success_rate", "num_tasks").orderBy("task_success_rate", ascending=True).show(10, truncate=False)

        if stages_df and stages_df.count() > 0:
            logger.info(" • Total Stages analyzed: %s", stages_df.count())
            print("\n=== TOP 10 STAGES BY DATA PROCESSED (ACROSS ALL CLUSTERS) ===")
            stages_df.select("cluster_name", "application_id", "stage_name", "total_data_processed_mb", "shuffle_data_mb").orderBy("total_data_processed_mb", ascending=False).show(10, truncate=False)

        if sql_df and sql_df.count() > 0:
            logger.info(" • Total SQL Queries analyzed: %s", sql_df.count())
            print("\n=== TOP 10 SQL QUERIES BY DURATION (ACROSS ALL CLUSTERS) ===")
            sql_df.select("cluster_name", "application_id", "sql_id", "description", "duration_minutes", "status").orderBy("duration_minutes", ascending=False).show(10, truncate=False)

        logger.info("🎉 Multi-cluster analysis complete! All data returned as DataFrames.")
        summary = {'total_clusters_analyzed': len([c for c in cluster_summaries if c['analysis_status'] == 'COMPLETED']), 'total_applications': len(all_applications_analysis), 'total_jobs': len(all_jobs_analysis), 'total_stages': len(all_stages_analysis), 'total_sql_queries': len(all_sql_analysis)}

        return {'cluster_summaries_df': cluster_summary_df, 'applications_df': apps_df, 'jobs_df': jobs_df, 'stages_df': stages_df, 'sql_df': sql_df, 'summary': summary}
    except Exception as e:
        logger.error("❌ Multi-cluster analysis failed: %s", str(e), exc_info=True)
        raise

try:
    results = main_analysis()
    cluster_summaries_df = results['cluster_summaries_df']
    applications_df = results['applications_df']
    jobs_df = results['jobs_df']
    stages_df = results['stages_df']
    sql_df = results['sql_df']
    analysis_summary = results['summary']

    print("\n" + "="*100 + "\n✨ EMR MULTI-CLUSTER SPARK HISTORY SERVER ANALYSIS COMPLETED! ✨\n" + "="*100)
    print(f"✅ Analyzed {analysis_summary['total_clusters_analyzed']} clusters\n✅ Analyzed {analysis_summary['total_applications']} applications\n✅ Analyzed {analysis_summary['total_jobs']} jobs\n✅ Analyzed {analysis_summary['total_stages']} stages\n✅ Analyzed {analysis_summary['total_sql_queries']} SQL queries")
    print("\n📊 DataFrames available for analysis:\n • cluster_summaries_df\n • applications_df\n • jobs_df\n • stages_df\n • sql_df\n • analysis_summary")
    print("\n💡 Example usage:\n applications_df.filter(col('duration_minutes') > 10).show()\n stages_df.groupBy('cluster_name').agg(sum('shuffle_data_mb')).show()\n sql_df.select('description', 'sql_raw_json').show(truncate=False)\n" + "="*100)

    if ENVIRONMENT == "prod" and S3_OUTPUT_PATH:
        logger.info("📦 Writing analysis results to S3: %s", S3_OUTPUT_PATH)
        try:
            if cluster_summaries_df and cluster_summaries_df.count() > 0:
                cluster_summaries_df.write.mode("overwrite").parquet(f"{S3_OUTPUT_PATH}/cluster_summaries/")
            if applications_df and applications_df.count() > 0:
                applications_df.write.mode("overwrite").parquet(f"{S3_OUTPUT_PATH}/applications/")
            if jobs_df and jobs_df.count() > 0:
                jobs_df.write.mode("overwrite").parquet(f"{S3_OUTPUT_PATH}/jobs/")
            if stages_df and stages_df.count() > 0:
                stages_df.write.mode("overwrite").parquet(f"{S3_OUTPUT_PATH}/stages/")
            if sql_df and sql_df.count() > 0:
                sql_df.write.mode("overwrite").parquet(f"{S3_OUTPUT_PATH}/sql/")
            logger.info("✅ All analysis results successfully written to S3")
        except Exception as s3_error:
            logger.error("❌ Failed to write results to S3: %s", str(s3_error), exc_info=True)
            print(f"⚠️ Warning: S3 write failed, but analysis completed successfully. Error: {str(s3_error)}")
    elif ENVIRONMENT == "dev":
        logger.info("🧪 Running in DEV mode - S3 output skipped")
        print("🧪 DEV Mode: Analysis results available in DataFrames only (no S3 output)")
    else:
        logger.info("🚫 PROD mode but no S3 output path specified - S3 output skipped")
        print("⚠️ PROD mode detected but no S3 output path provided - results available in DataFrames only")
except Exception as e:
    print(f"\n❌ ANALYSIS FAILED: {str(e)}\nPlease check the logs above for detailed error information.")


2025-08-19 00:03:30,816 - INFO - 🚀 Starting EMR Spark History Server Analysis
2025-08-19 00:03:30,817 - INFO - 🔭 Multi-cluster discovery mode - searching for EMR clusters
2025-08-19 00:03:30,819 - INFO - 🔍 Discovering EMR clusters in region: us-east-1
2025-08-19 00:03:30,820 - INFO -  States: ['STARTING', 'BOOTSTRAPPING', 'RUNNING', 'WAITING', 'TERMINATING', 'TERMINATED', 'TERMINATED_WITH_ERRORS']
2025-08-19 00:03:30,820 - INFO -  Name filter: None
2025-08-19 00:03:30,820 - INFO -  Max clusters: 10
2025-08-19 00:03:30,820 - INFO -  Created After: None
2025-08-19 00:03:30,820 - INFO -  Created Before: None
2025-08-19 00:03:31,050 - INFO - ✅ Discovered 10 clusters
2025-08-19 00:03:31,051 - INFO -  1. spark-canary-job-aws @ 2025-08-18T15:02:04.727576+00:00 :: Airflow (j-2C9L9Y9WYQ6H8) - TERMINATED
2025-08-19 00:03:31,051 - INFO -  2. spark-canary-job-aws @ 2025-08-18T15:02:04.727576+00:00 :: Airflow (j-2F9HRWX3WB2KP) - TERMINATED
2025-08-19 00:03:31,051 - INFO -  3. spark-canary-job-aws @


📊 CLUSTER ANALYSIS SUMMARY


2025-08-19 00:08:45,056 - INFO - Received command c on object id p0
2025-08-19 00:08:46,056 - INFO - Received command c on object id p0


Cluster Summary (10 clusters):
+---------------+------------------------------------------------------------------+----------+---------------+-----------------------------------------------------------------------------------------+------------------+----------+------------+-----------------+
|cluster_id     |cluster_name                                                      |status    |analysis_status|status_details                                                                           |total_applications|total_jobs|total_stages|total_sql_queries|
+---------------+------------------------------------------------------------------+----------+---------------+-----------------------------------------------------------------------------------------+------------------+----------+------------+-----------------+
|j-310AFAY6L57Q |Gautam: UserId Seller Agg                                         |TERMINATED|NO_APPLICATIONS|No applications found in Spark History Server                        

2025-08-19 00:08:46,471 - INFO -  • Total Applications analyzed: 7



=== TOP 10 APPLICATIONS BY DURATION (ACROSS ALL CLUSTERS) ===


2025-08-19 00:08:46,810 - INFO - 🎉 Multi-cluster analysis complete! All data returned as DataFrames.
2025-08-19 00:08:46,812 - INFO - 🧪 Running in DEV mode - S3 output skipped


+------------------------------------------------------------------+---------------------------------------------------+----------------+-----+----------------+
|cluster_name                                                      |application_name                                   |duration_minutes|cores|efficiency_score|
+------------------------------------------------------------------+---------------------------------------------------+----------------+-----+----------------+
|spark-canary-job-aws @ 2025-08-18T15:02:04.727576+00:00 :: Airflow|CanaryPipeline-class-jobs.dataproc.canary.CanaryJob|0.0             |0    |0.0             |
|Gautam: CleanRoomUserIdSellerAgg                                  |CleanRoomUserIdSellerAgg                           |0.0             |0    |0.0             |
|spark-canary-job-aws @ 2025-08-18T15:02:04.727576+00:00 :: Airflow|CanaryPipeline-class-jobs.dataproc.canary.CanaryJob|0.0             |0    |0.0             |
|spark-canary-job-aws @ 2025-08-15

### Table Exploration

In [0]:
display(analysis_summary)

{'total_clusters_analyzed': 7,
 'total_applications': 7,
 'total_jobs': 0,
 'total_stages': 0,
 'total_sql_queries': 0}

In [0]:
display(cluster_summaries_df)

2025-08-19 00:08:47,056 - INFO - Received command c on object id p0


cluster_id,cluster_name,status,analysis_status,status_details,total_applications,total_jobs,total_stages,total_sql_queries
j-310AFAY6L57Q,Gautam: UserId Seller Agg,TERMINATED,NO_APPLICATIONS,No applications found in Spark History Server,0,0,0,0
j-160Y1JZF990JG,spark-canary-job-aws @ 2025-08-15T21:42:36.417304+00:00 :: Airflow,TERMINATED,COMPLETED,applications: OK; jobs: FAILED; sql: FAILED,1,0,0,0
j-2C9L9Y9WYQ6H8,spark-canary-job-aws @ 2025-08-18T15:02:04.727576+00:00 :: Airflow,TERMINATED,COMPLETED,applications: OK; jobs: FAILED; sql: FAILED,1,0,0,0
j-1ILKHEQ510RAI,Gautam: CleanRoomUserIdSellerAgg,TERMINATED,COMPLETED,applications: OK; jobs: FAILED; sql: FAILED,1,0,0,0
j-1KPXX0HR3U0BN,Gautam: CleanRoomUserIdSellerAgg,TERMINATED,COMPLETED,applications: OK; jobs: FAILED; sql: FAILED,1,0,0,0
j-137KTJ1T8J5TS,spark-canary-job-aws @ 2025-08-15T22:24:16.775314+00:00 :: Airflow,TERMINATED,COMPLETED,applications: OK; jobs: FAILED; sql: FAILED,1,0,0,0
j-2F9HRWX3WB2KP,spark-canary-job-aws @ 2025-08-18T15:02:04.727576+00:00 :: Airflow,TERMINATED,COMPLETED,applications: OK; jobs: FAILED; sql: FAILED,1,0,0,0
j-1VJLYTUUO02M0,Gautam: UserId Seller Agg,TERMINATED,COMPLETED,applications: OK; jobs: FAILED; sql: FAILED,1,0,0,0
j-2O70N006Y1DEP,Gautam: CleanRoomUserIdSellerAgg,TERMINATED,FAILED,Cluster analysis error: Persistent App UI did not become ready after waiting 300 seconds.,0,0,0,0
j-38XT10OQKQR6U,Gautam: UserId Seller Agg,TERMINATED,FAILED,Cluster analysis error: Persistent App UI did not become ready after waiting 300 seconds.,0,0,0,0


In [0]:
display(applications_df)

cluster_id,cluster_name,application_id,application_name,duration_ms,duration_minutes,start_time,end_time,spark_version,cores,memory_per_executor_mb,max_cores,core_hours,efficiency_score
j-160Y1JZF990JG,spark-canary-job-aws @ 2025-08-15T21:42:36.417304+00:00 :: Airflow,application_1755294357227_0001,CanaryPipeline-class-jobs.dataproc.canary.CanaryJob,0,0.0,null,null,Unknown,0,0,0,0.0,0.0
j-2C9L9Y9WYQ6H8,spark-canary-job-aws @ 2025-08-18T15:02:04.727576+00:00 :: Airflow,application_1755540700327_0001,CanaryPipeline-class-jobs.dataproc.canary.CanaryJob,0,0.0,null,null,Unknown,0,0,0,0.0,0.0
j-1ILKHEQ510RAI,Gautam: CleanRoomUserIdSellerAgg,application_1754586783613_0001,CleanRoomUserIdSellerAgg,0,0.0,null,null,Unknown,0,0,0,0.0,0.0
j-1KPXX0HR3U0BN,Gautam: CleanRoomUserIdSellerAgg,application_1754583978065_0001,CleanRoomUserIdSellerAgg,0,0.0,null,null,Unknown,0,0,0,0.0,0.0
j-137KTJ1T8J5TS,spark-canary-job-aws @ 2025-08-15T22:24:16.775314+00:00 :: Airflow,application_1755296864657_0001,CanaryPipeline-class-jobs.dataproc.canary.CanaryJob,0,0.0,null,null,Unknown,0,0,0,0.0,0.0
j-2F9HRWX3WB2KP,spark-canary-job-aws @ 2025-08-18T15:02:04.727576+00:00 :: Airflow,application_1755529524344_0001,CanaryPipeline-class-jobs.dataproc.canary.CanaryJob,0,0.0,null,null,Unknown,0,0,0,0.0,0.0
j-1VJLYTUUO02M0,Gautam: UserId Seller Agg,application_1754533647407_0001,CleanRoomExplodedDealAvailsAggTransformer,0,0.0,null,null,Unknown,0,0,0,0.0,0.0


In [0]:
display(jobs_df)

In [0]:
display(stages_df)

2025-08-19 00:08:48,056 - INFO - Received command c on object id p0


In [0]:
display(sql_df)